In [1]:
import numpy as np
import pandas as pd
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
import re
import os

# os.environ["CUDA_VISIBLE_DEVICES"]="-1"    
import tensorflow as tf
# from sklearn.model_selection import train_test_split
# gpus = tf.config.experimental.list_physical_devices('GPU')
# tf.config.experimental.set_memory_growth(gpus[0], True)

# tf.test.is_gpu_available(
#     cuda_only=False, min_cuda_compute_capability=None
# )
# gpu = tf.config.list_physical_devices('GPU')
# tf.config.experimental.set_memory_growth(gpu[0],True)

# print(tf.config.list_physical_devices('GPU'))
# tf.test.is_built_with_cuda()

In [2]:
# gpus = tf.config.experimental.list_physical_devices('GPU')
# tf.config.experimental.set_memory_growth(gpus[0], True)

In [3]:
# df_train = pd.read_csv('train.csv')
# df_val = pd.read_csv("val.csv")
# text_data = pd.concat([df_train,df_val])
# text_data

1. Data Cleaning and Processing

In [4]:
text_data = pd.read_csv("val.csv")

def clean_data(text):
    text = text.lower()
    text = re.sub(r"[-=+*\"#@!$%^&()`<>\[\]]","",text)
    text = re.sub(r"i'm","i am",text)
    text = re.sub(r"he's","he is",text)
    text = re.sub(r"she's","she is",text)
    text = re.sub(r"it's","it is",text)
    text = re.sub(r"they're","they are",text)
    text = re.sub(r"there're","there are",text)
    text = re.sub(r"there's","there is",text)
    text = re.sub(r"how're","how are",text)
    text = re.sub(r"what're","what are",text)
    text = re.sub(r"where're","where ",text)
    text = re.sub(r"who're","who are",text)
    text = re.sub(r"that're","that are",text)
    text = re.sub(r"when're","when are",text)
    text = re.sub(r"how's","how is",text)
    text = re.sub(r"what's","what is",text)
    text = re.sub(r"where's","where is",text)
    text = re.sub(r"who's","who is",text)
    text = re.sub(r"that's","that is",text)
    text = re.sub(r"when's","when is",text)
    text = re.sub(r"won't","would not",text)
    text = re.sub(r"nt't","can not",text)
    text = re.sub(r"\'bout'","about",text)
    text = re.sub(r"\'till'","untill",text)
    text = re.sub(r"\'ll","will",text)
    text = re.sub(r"\'ve","have",text)
    text = re.sub(r"\'re","are",text)
    text = re.sub(r"\'d","would",text)
    text = re.sub(r"\."," ",text)
    text = re.sub(r"\,"," ",text)
    text = re.sub(r"\!"," ",text)
    text = re.sub(r"\?"," ",text)
    text = re.sub(r"\;"," ",text)
    text = re.sub(r"\:"," ",text)
    return text

    
text_data['text'] = text_data['text'].apply(clean_data)
text_data['augmented_text'] = text_data['augmented_text'].apply(clean_data)

# # text_data.to_csv('data.csv')


In [5]:
max_length = 10

text_tokenizer = Tokenizer()
text_tokenizer.fit_on_texts(text_data['text'])
text_word_index = text_tokenizer.word_index
test_sequences = text_tokenizer.texts_to_sequences(text_data['text'])
test_padded = pad_sequences(test_sequences, maxlen=max_length, padding='post',truncating='post')

g_text_tokenizer = Tokenizer()
g_text_tokenizer.fit_on_texts(text_data['augmented_text'])
g_text_word_index = g_text_tokenizer.word_index
train_sequences = g_text_tokenizer.texts_to_sequences(text_data['augmented_text'])
train_padded = pad_sequences(train_sequences, maxlen=max_length, padding='post',truncating='post')




2 Model Building

In [6]:

g_vocab_size = len(g_text_word_index)+1
vocab_size = len(text_word_index)+1



model = tf.keras.Sequential([
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=True), input_shape=(10,1)),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(64, activation='relu')),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(vocab_size,activation='softmax'))
])
# learning_rate = 2
optimizer = tf.keras.optimizers.RMSprop()
optimizer = tf.keras.mixed_precision.LossScaleOptimizer(optimizer)
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=['accuracy'])






In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirectiona  (None, 10, 64)           8704      
 l)                                                              
                                                                 
 time_distributed (TimeDistr  (None, 10, 64)           4160      
 ibuted)                                                         
                                                                 
 dropout (Dropout)           (None, 10, 64)            0         
                                                                 
 time_distributed_1 (TimeDis  (None, 10, 47127)        3063255   
 tributed)                                                       
                                                                 
Total params: 3,076,119
Trainable params: 3,076,119
Non-trainable params: 0
______________________________________________

In [8]:
test_padded.shape

(153728, 10)

# To do just change the epoch to 10, 20, 50, 100

In [9]:
epoch = 
temp_test_padded = test_padded.reshape(test_padded.shape[0], test_padded.shape[1],1)
# train_padded = train_padded.reshape(train_padded.shape[0], train_padded.shape[1], 1)

model.fit(train_padded, temp_test_padded, epochs=epoch,verbose=1,batch_size=512)

# try:
#     with tf.device(gpu[0]):
# except RuntimeError as e:
#       print(e)

Epoch 1/5
301/301 [==============================] - 1054s 3s/step - loss: 5.9859 - accuracy: 0.3034
Epoch 2/5
301/301 [==============================] - 1049s 3s/step - loss: 5.2281 - accuracy: 0.3504
Epoch 3/5
301/301 [==============================] - 1049s 3s/step - loss: 5.0809 - accuracy: 0.3708
Epoch 4/5
301/301 [==============================] - 1050s 3s/step - loss: 4.9808 - accuracy: 0.3789
Epoch 5/5
301/301 [==============================] - 1048s 3s/step - loss: 4.9049 - accuracy: 0.3839


In [10]:
#save model
model.save('5_epoch_accuracy_0.3839_bidirectional_lstm_rnn')
#load model
n_m= tf.keras.models.load_model('5_epoch_accuracy_0.3839_bidirectional_lstm_rnn')

INFO:tensorflow:Assets written to: 5_epoch_accuracy_0.3839_bidirectional_lstm_rnn\assets


INFO:tensorflow:Assets written to: 5_epoch_accuracy_0.3839_bidirectional_lstm_rnn\assets
